<a href="https://colab.research.google.com/github/matheusdasilvaoliveira/jogo-de-sucesso/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "games.json"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "fronkongames/steam-games-dataset",
  file_path
)

print("First 5 records:", df)

/tmp/ipython-input-3644561937.py:6: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'steam-games-dataset' dataset.
First 5 records:                                                                     20200    \
name                                                       Galactic Bowling   
release_date                                                   Oct 21, 2008   
required_age                                                              0   
price                                                                 19.99   
dlc_count                                                                 0   
detailed_description      Galactic Bowling is an exaggerated and stylize...   
about_the_game            Galactic Bowling is an exaggerated and stylize...   
short_description         Galactic Bowling is an exaggerated and stylize...   
reviews                                                                       
header_image              https://cdn.akamai.steamstatic.com/steam/apps/...   
website                                 

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "games.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "antonkozyriev/game-recommendations-on-steam",
  file_path
)

print(df.columns)
print("First 5 records:", df)

/tmp/ipython-input-2606905278.py:10: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'game-recommendations-on-steam' dataset.
Index(['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating',
       'positive_ratio', 'user_reviews', 'price_final', 'price_original',
       'discount', 'steam_deck'],
      dtype='object')
First 5 records:         app_id                                      title date_release   win  \
0        13500          Prince of Persia: Warrior Within™   2008-11-21  True   
1        22364                    BRINK: Agents of Change   2011-08-03  True   
2       113020               Monaco: What's Yours Is Mine   2013-04-24  True   
3       226560                         Escape Dead Island   2014-11-18  True   
4       249050                    Dungeon of the ENDLESS™   2014-10-27  True   
...        ...                                        ...          ...   ...   
50867  2296380  I Expect You To Die 3: Cog in the Machine   2023-09-28  True   
50868  1272080                                   PAYDA